In [1]:
import json
import numpy as np
import pandas as pd
import ipaddress

In [2]:
#HELPERS

#returns if the ip is private or not, takes Str as inpt
def is_private_ip(ip):
    ip_address = ipaddress.ip_address(ip)
    return ip_address.is_private

#returns if the hop has value or not, takes dict as inpt --> list of valid inpts
def has_value(hop):
    out = []
    rtts = []
    rtt = []
    for route in hop['result']:
        if 'x' in route or 'rtt' not in route:
            continue
        out.append(route['from'])
        rtts.append(route['rtt'])
        
    
    #check for duplicates
    new_out = np.unique(out).tolist()
    #find the rtt average for the same values
    if len(new_out) == 1:
        rtt.append(np.average(rtts))
    if len(new_out) > 1:
        #check which ones are unqiue
        for val in new_out:
            #looking for the rtts to average and which not to when values are different
            indices = [i for i, x in enumerate(out) if x == val]
            total = 0
            for index in indices:
                total += rtts[index]
            rtt.append(total/len(indices))
            
    return new_out, rtt

#checks if the IP belongs to the IXP range
#Enter the list of Prefixes that you want to check against, control is the list of all the IXP prefixes
#Takes in a list of prefixes(str) and a single ip(str)
def in_IXP(control,ip):
    for prefix in control:
        if ipaddress.ip_address(ip) in ipaddress.ip_network(prefix):
            return True
    return False

#takes in the traceroute(list of all hops) and finds the last valid hop, (hops[{},{},....])
#Returns at least 1 value back
def last_valid_hop(hops,prefix):
    for i in range(len(hops)-1,-1,-1):
        hop = hops[i]
        values,rtt = has_value(hop)
        
        valid_values = []
        
        # Upper bound of 3 values at max
        for value in values:
            if is_private_ip(value) == False and in_IXP(prefix,value) == False:
                valid_values.append(value)
                

        if len(valid_values) != 0:
            return (i,valid_values,len(hops)),rtt
    
    #-1,[] --> No valid hop
    return (-1,[],len(hops)),rtt

#Checks if the destresponse is true, and if it is returns the last hop IP, else empty list
def dest_reponse(response,hops):
    dest = []
    rtt = 0
    if response:
        if 'result' in hops[-1]:
            for pckt in hops[-1]['result']:
                if 'rtt' in pckt:
                    rtt += pckt['rtt']
            
            rtt /= 3 #There is only 1 destination at the end of the route
            
            if 'from' in hops[-1]:
                dest.append(hops[-1]['from'])
    return dest,rtt

In [3]:
#Json aggregator, takes a file to be read(str path) and list of prefixes(str[])
def loader_reader(file,prefixes):
    dest = []
    dest_name = []
    source = []
    l_valid_hop = []
    rtts = []
    dst_responses = []
    dst_rtts = []
    success = []

    
    #'x' --> no data found
    with open(file,'r') as f:
        probe_data = json.load(f)
        
        for probe in probe_data:
            #Checks for the key values, just in case
            if 'dst_addr' in probe:
                dest.append(probe['dst_addr'])
                temp_dest_pointer = probe['dst_addr']
            else:
                dest.append('x')
                
            if 'dst_name' in probe:
                dest_name.append(probe['dst_name'])
            else:
                dest_name.append('x')
                
            if 'src_addr' in probe:
                source.append(probe['src_addr'])
            else:
                source.append('x')
                
                
            if 'result' in probe:
                
                #checking for the last valid hop on the trace from the probe and it's rtt
                value,rtt = last_valid_hop(probe['result'],prefixes)
                if len(value) != 0:
                    l_valid_hop.append(value)
                else:
                    l_valid_hop.append('x')
                rtts.append(rtt)
                
                #Checking if the traceroute was fully successful
                if 'destination_ip_responded' in probe:
                    response = probe['destination_ip_responded']
                    success.append(response)
                    
                    dst_response_ip,dst_rtt = dest_reponse(response,probe['result'])
                    
                    
                #Destination that responded is not the same as the last hop(shouldn't happen at all but just in case)
                    if response or len(dst_response_ip) == 0 or dst_response_ip[0] != temp_dest_pointer:
                        dst_responses.append('x')
                        dst_rtts.append('x')
                        
                        
                    
                    else:
                        dst_responses.append(dst_response_ip[0])
                        dst_rtts.append(dst_rtt)
                
                else:
                    dst_responses.append('x')
                    dst_rtts.append('x')
                    success.append(False)
                    
            else:
                l_valid_hop.append('x')
                rtts.append(-1)
                

    return source,dest,dest_name,l_valid_hop,rtts,success,dst_responses,dst_rtts
            

In [4]:
def create_csv(path,fileName,prefixes):
    file = path+'/'+fileName+'.json'
    source,dest,dest_name,l_valid_hop,rtts,success,dst_responses,dst_rtts = loader_reader(file,prefixes)
    data = {'Source_IP':source,'Destination_Name':dest_name,'Destination_IP':dest,'Last_Valid_Hop':l_valid_hop,'RTT':rtts,'Success?':success,'Destination_IP_responded':dst_responses,'Destination_RTT':dst_rtts}
    df = pd.DataFrame(data)
    df.to_csv(path+'/out.csv', index=False)

In [5]:
#Make a list of all the IXP Prefixes, then run create CSV for all of the traceroutes, then compare the IPs in traceroutes with
#respective IPs in the bdrmapit
def create_list(path):
    data = pd.read_csv(path)
    prefixes = data['prefixes'].unique()
    output = []
    for prefix in prefixes:
        temp = prefix.replace("'",'"')
        parsed_pfx = json.loads(temp)
        output.append(parsed_pfx['ipv4'][0])
    
    return output

In [6]:
#Gathering IXP prefixes
out1 = create_list('/Users/nishantacharya/Desktop/bdrmapit_updated/289I/CSV_files/IXP_ASN.csv')
out2 = create_list('/Users/nishantacharya/Desktop/bdrmapit_updated/289I/CSV_files/SK_IXP_ASN.csv')

prefix = []
prefix.extend(out1)
prefix.extend(out2)
prefix.append('185.1.158.0/24')

In [7]:
prefix

['185.1.48.0/24',
 '185.0.0.0/24',
 '212.81.56.0/24',
 '185.1.202.0/24',
 '86.104.125.0/24',
 '185.1.128.0/24',
 '185.1.251.0/24',
 '185.1.150.0/24',
 '185.1.179.0/24',
 '185.1.158.0/24',
 '185.1.204.0/24',
 '194.30.187.0/24',
 '185.1.234.0/24',
 '192.108.145.0/24',
 '192.108.148.0/24',
 '185.1.158.0/24']

In [8]:
#Creating last hop csvs for all the traceroutes
#Romania
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68324855/1','RIPE-Atlas-measurement-68324855',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68324979/2','RIPE-Atlas-measurement-68324979',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325036/3','RIPE-Atlas-measurement-68325036',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325052/4','RIPE-Atlas-measurement-68325052',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325061/5','RIPE-Atlas-measurement-68325061',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325073/6','RIPE-Atlas-measurement-68325073',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325205/7','RIPE-Atlas-measurement-68325205',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325211/8','RIPE-Atlas-measurement-68325211',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325216/9','RIPE-Atlas-measurement-68325216',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325221/10','RIPE-Atlas-measurement-68325221',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325255/11','RIPE-Atlas-measurement-68325255',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/RIPE-Atlas-measurement-68325263/12','RIPE-Atlas-measurement-68325263',prefix)

In [9]:
#Slovakia
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281139/1','RIPE-Atlas-measurement-68281139',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281145/2','RIPE-Atlas-measurement-68281145',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281156/3','RIPE-Atlas-measurement-68281156',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281200/4','RIPE-Atlas-measurement-68281200',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281414/5','RIPE-Atlas-measurement-68281414',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281436/6','RIPE-Atlas-measurement-68281436',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281442/7','RIPE-Atlas-measurement-68281442',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281461/8','RIPE-Atlas-measurement-68281461',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281481/9','RIPE-Atlas-measurement-68281481',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281526/10','RIPE-Atlas-measurement-68281526',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281530/11','RIPE-Atlas-measurement-68281530',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281543/12','RIPE-Atlas-measurement-68281543',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281547/13','RIPE-Atlas-measurement-68281547',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/RIPE-Atlas-measurement-68281551/14','RIPE-Atlas-measurement-68281551',prefix)

In [12]:
#Turkey
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/T/Turkey/RIPE-Atlas-measurement-68281129/1','RIPE-Atlas-measurement-68281129',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/T/Turkey/RIPE-Atlas-measurement-68281144/2','RIPE-Atlas-measurement-68281144',prefix)
#create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/T/Turkey/RIPE-Atlas-measurement-68281603/3','RIPE-Atlas-measurement-68281603',prefix)
create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/bdrmapit/bdrmapit/traceroutes/T/Turkey/RIPE-Atlas-measurement-68539630/5','/RIPE-Atlas-measurement-68539630',prefix)
create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/bdrmapit/bdrmapit/traceroutes/T/Turkey/RIPE-Atlas-measurement-68539643/4','RIPE-Atlas-measurement-68539643',prefix)
create_csv('/Users/nishantacharya/Desktop/bdrmapit_updated/bdrmapit/bdrmapit/traceroutes/T/Turkey/RIPE-Atlas-measurement-68539655/6','RIPE-Atlas-measurement-68539655',prefix)

In [65]:
import os

In [144]:
#Helper
#Check if the routers in last hop are a border router, all are arrays
#lastHopRouters(tuple,size 3, (hop no, ip,totalhops)), brdrRouters, list of routers, Success boolean list
def check_AS_border(lastHopRouters,brdrRouters,lastHopSuccess):
    hit = []
    for i in range(len(lastHopRouters)):
        success = lastHopSuccess[i]
        router =  eval(lastHopRouters[i])[1]
        #If the route ends, no need to check
        if success:
            hit.append(True)
        elif len(router) > 0 and router[0] in brdrRouters:
            hit.append(True)
        else:
            hit.append(False)
    return hit

def is_hidden_file(filename):
    return filename.startswith(".")

#Check how many routers in the last valid hop csv are border routers and mark them as complete
def analyze_lastHops(root_dir):
    for entry in os.listdir(root_dir):
        upperPath = os.path.join(root_dir,entry)
        if os.path.isdir(upperPath):
            for mid in os.listdir(upperPath):
                fullPath = os.path.join(upperPath,mid)
                if os.path.isdir(fullPath):
                    lastHops = pd.read_csv(fullPath+'/out.csv')
                    brdrHops = pd.read_csv(fullPath+'/bdrmapit_output.csv')
                    hits = check_AS_border(lastHops['Last_Valid_Hop'],brdrHops['addr'].unique(),lastHops['Success?'])
                    lastHops['possibly_reached?'] = hits
                    lastHops.to_csv(fullPath+'/final_out.csv')
        

In [146]:
analyze_lastHops('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/T/Turkey/')
analyze_lastHops('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/R/Romania/')
analyze_lastHops('/Users/nishantacharya/Desktop/bdrmapit_updated/traceroutes/S/Slovakia/')